# [Using EasyOCR with pretrained YOLO for License Plate Recognition](https://www.youtube.com/watch?v=9KnARRqbkzY)

---

This notebook demonstrates how to use EasyOCR in combination with a pretrained YOLO model to perform license plate recognition. The YOLO model is used to detect license plates in images, and EasyOCR is then used to read the text from the detected plates.


Before running a notebook for the first time, install the required dependencies with:

```bash
	make requirements-nb
```


In [ ]:
from civicpulseML.config import BASE_DIR, CUDA_AVAILABLE, MODELS_DIR, RAW_DIR

## Importing dataset from Roboflow

---
